In [14]:
import nibabel as nib
import numpy as np
import pandas as pd
import os

In [16]:
DATA_FOLDER = "./data/BET_BSE_DATA/"

In [18]:
img=nib.load(DATA_FOLDER+'files/IXI662-Guys-1120-T1_bse_less_s46_r1.nii.gz')
print('Shape of image=',img.shape)

Shape of image= (256, 256, 150)


In [19]:
mri_image_files = os.listdir(DATA_FOLDER+'/files')

In [20]:
print(mri_image_files[0:10])

['IXI369-Guys-0924-T1_bse_high_s75_r2.nii.gz', 'IXI270-Guys-0847-T1_bet_88.nii.gz', 'IXI333-IOP-0926-T1_bse_default.nii.gz', 'IXI483-HH-2177-T1_bet_12.nii.gz', 'IXI464-IOP-1029-T1_bet_08.nii.gz', 'IXI205-HH-1649-T1_bet_73.nii.gz', 'IXI544-HH-2395-T1_bse_less_s51_r1.nii.gz', 'IXI033-HH-1259-T1_bet_76.nii.gz', 'IXI260-HH-1805-T1_bet_12.nii.gz', 'IXI241-Guys-0833-T1_bse_high_s74_r2.nii.gz']


In [21]:
labels = pd.read_csv(DATA_FOLDER+'Label_file.csv')

In [22]:
labels.head()

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI369-Guys-0924-T1_bet_03.nii,Yes,No
1,IXI448-HH-2393-T1_bet_07.nii,Yes,No
2,IXI252-HH-1693-T1_bet_08.nii,Yes,No
3,IXI188-Guys-0798-T1_bet_17.nii,Yes,No
4,IXI182-Guys-0792-T1_bet_17.nii,Yes,No
